In [25]:
!unzip tokendataset_.zip -d tokendataset_

Archive:  tokendataset_.zip
  inflating: tokendataset_/dataset_dict.json  
   creating: tokendataset_/test/
  inflating: tokendataset_/test/data-00000-of-00001.arrow  
  inflating: tokendataset_/test/dataset_info.json  
  inflating: tokendataset_/test/state.json  
   creating: tokendataset_/train/
  inflating: tokendataset_/train/data-00000-of-00001.arrow  
  inflating: tokendataset_/train/dataset_info.json  
  inflating: tokendataset_/train/state.json  
   creating: tokendataset_/validation/
  inflating: tokendataset_/validation/data-00000-of-00001.arrow  
  inflating: tokendataset_/validation/dataset_info.json  
  inflating: tokendataset_/validation/state.json  


In [7]:
model_checkpoint = "facebook/galactica-125M"
lr = 1e-3
batch_size = 128
num_epochs = 3
dataset = load_dataset("math_qa")
dataset = dataset.remove_columns(['options', 'correct', 'linear_formula', 'category'])
model = OPTForCausalLM.from_pretrained(model_checkpoint)
print("dataset download success!\n")
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-1.3b", use_fast=True)
tokenizer.pad_token_id = 1
tokenizer.pad_token = "<pad>"
tokenizer.padding_side = "left"

# setup truncation
tokenizer.truncation_side = "left"

# setup special tokens
tokenizer.bos_token_id = 0
tokenizer.bos_token = "<s>"

tokenizer.eos_token_id = 2
tokenizer.eos_token = "</s>"

tokenizer.unk_token = "<unk>"
tokenizer.unk_token_id = 3
SPLIT_MARKER = f"SPL{1}T-TH{1}S-Pl3A5E"
def _insert_split_marker(m: re.Match):
    """
    Applies split marker based on a regex match of special tokens such as
    [START_DNA].
    Parameters
    ----------
    n : str
        Input text to split
    Returns
    ----------
    str - the text with the split token added
    """
    start_token, _, sequence, end_token = m.groups()
    sequence = re.sub(r"(.)", fr"{SPLIT_MARKER}\1", sequence, flags=re.DOTALL)
    return f"{start_token}{sequence}{SPLIT_MARKER}{end_token}"
def escape_custom_split_sequence(text):
    return re.compile(r"(\[START_(DNA|SMILES|I_SMILES|AMINO)])(.*?)(\[END_\2])").sub(_insert_split_marker, text)
def _tokenize(input_text, new_doc) -> torch.LongTensor:
    """
    Apply custom preprocessing to input texts and tokenize them.
    Returns
    -------
        input_text : list[str]
            Texts to be tokenized
        new_doc : bool
            If True, prepends the end-of-document (</s>) token to each sequence and fixes
            padding.
    """
    texts = []
    for text in input_text:
        text = escape_custom_split_sequence(text)
        if not text:
            warnings.warn(
                "Found an empty input text. Changing to end-of-document token instead.",
                UserWarning
            )
            text = tokenizer.eos_token
        texts.append(text)

    if new_doc:
        pad_token = tokenizer.pad_token
        texts = [pad_token + t for t in texts]

    encoded = tokenizer(
        texts,
        padding="longest",
        max_length=500,
        truncation=True
    )
    # print(encoded)
    # context_tokens = encoded["input_ids"]
    # input_v = torch.LongTensor(context_tokens).to(model.device)

    # if new_doc:
    #     input_v[input_v[:, 0] == tokenizer.pad_token_id, 0] = tokenizer.eos_token_id
    return encoded
def combine_features(example):
  problem = example['Problem']
  rationale = example['Rationale']
  annotated_formula = example["annotated_formula"]
  combine_feature = [f"{problem[i]} <work> {rationale[i]} {annotated_formula[i]}" for i in range(len(problem))]
  return {"text":combine_feature}
  #return example['Problem'] + ' <work> ' + example['Rationale'] + " " + example["annotated_formula"]
dataset = dataset.map(combine_features, batched=True, remove_columns=['Problem', 'Rationale', 'annotated_formula'], num_proc=4)

def tokenize(example):
  tokinized = [_tokenize(example['text'][i], False) for i in range(len(example['text']))]
  tok_keys = tokinized[0].keys()
  print(tok_keys)
  result = {}
  for i in tok_keys:
    for j in range(len(tokinized)):
      result[i] = result.get(i, []) + tokinized[j][i]
  return result#_tokenize(example['text'], False)
tokendataset = dataset.map(tokenize, batched=True, remove_columns=['text'], num_proc=4)
tokendataset.save_to_disk("tokendataset")

In [26]:
datasets = load_from_disk("./tokendataset_")

In [27]:
datasets['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13758571
})

In [36]:
!python3 run_mathqa.py

imported success!

DatasetDict({                                                                   
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 101045
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10188
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 15152
    })
})
dataset download success!

model download success!

training_args success!

Using cuda_amp half precision backend
trainer success!

The following columns in the training set don't have a corresponding argument in `OPTForCausalLM.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `OPTForCausalLM.forward`,  you can safely ignore this message.
/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: Thi